### Carzam
Trying to do transfer learning on the Vgg16 model for predicting car make/model.

In [1]:
import os

import keras
from keras.preprocessing import image

from matplotlib import pyplot as plt
import numpy as np
from numpy.random import permutation
np.set_printoptions(precision=4, linewidth=100)

Using Theano backend.


In [2]:
# Contains some utilty functions
# TODO Figure out what "reload" does
import utils; reload(utils)
from utils import plots
# Contains the trained Vgg16 model (2014 winner of ImageNet)
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [3]:
carzam_path = "../data/carzam/"
# carzam_path = "../data/carzam/sample/"
# Path containing trained models
carzam_model_path = carzam_path + 'models/'
if not os.path.exists(carzam_model_path): os.mkdir(carzam_model_path)

In [4]:
# If running out of memory or using an older GPU, decrease this number
batch_size = 4

In [5]:
vgg = Vgg16()
# Grab a few images at a time for training and validation
# batches = vgg.get_batches(path+'train', batch_size=batch_size)
# val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

MemoryError: 

In [ ]:
imgs, labels = next(batches)

In [ ]:
plots(imgs, titles=labels)

In [ ]:
vgg.finetune(batches)

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=5)

In [ ]:
vgg.model.save_weights(model_path+'finetune1.h5')